# Azure AI Models Quotas and Usages Discovery Recipe

This Jupyter notebook serves as a guide for users looking to understand how to discover Azure AI Models, Quotas and subscription Usages across regions.

## Pre-requisites

Authenticate to Azure CLI

```console
az login --use-device-code
```

## Get Azure Subscription

In [15]:
import requests
import json
import pandas as pd
from datetime import datetime
from azure.mgmt.subscription import SubscriptionClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
sub_client = SubscriptionClient(credential)
subscription = next(sub_client.subscriptions.list(), None)
if not subscription:
    raise Exception("Authenticate using the az cli")
subscriptionId = subscription.subscription_id

print(f"Using subscription {"x" * 8  + "-" + ("x" * 4 + "-")*3}{subscriptionId[-12:]}")

Using subscription xxxxxxxx-xxxx-xxxx-xxxx-2f5e9419c1cd


## Get API token and headers

In [ ]:
token = credential.get_token('https://management.azure.com/.default')
headers = {'Authorization': 'Bearer ' + token.token}

## Locations API: Discover regions and geography groups

### Regions

In [ ]:
def get_regions():
    locations_request = f"https://management.azure.com/subscriptions/{subscriptionId}/locations?api-version=2021-04-01"
    response = requests.get(locations_request, headers=headers)
    data = json.loads(response.text)
    return data["value"]

regions = get_regions()

def get_region_names(regions):
    return set([r['name'] for r in regions])

regions_list = get_region_names(regions)
print(f"{len(regions_list)} regions:")
print(", ".join(regions_list))

### Geography groups

In [ ]:
def get_geography_groups(regions):
    return set([region['metadata']['geographyGroup'] for region in regions if 'geographyGroup' in region['metadata']])

geography_groups = get_geography_groups(regions)
print(", ".join(geography_groups))

### Let's focus on US regions

In [ ]:
def filter_by_geography_group(regions, *args):
    return filter(lambda r: r['metadata']['geographyGroup'] in args, [r for r in regions if 'geographyGroup' in r['metadata']])
regions_list = get_region_names(filter_by_geography_group(regions, 'US'))
print(", ".join(regions_list))

### Helper to parallelize calls per region

Most of the API calls we need to make going forward are region-specific, and we aim to aggregate results across multiple regions. Depending on the number of regions being queried, this could involve up to 90 regions. The following helper function enables parallel execution of queries for each region.

In [ ]:
def parallel_map(fn, *iterables, executor=None, **kwargs):
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from tqdm import tqdm

    """
    Equivalent to executor.map(fn, *iterables), but displays a tqdm-based progress bar.
    
    **kwargs is passed to tqdm.
    """
    with executor if executor else ThreadPoolExecutor(max_workers=len(regions_list)) as ex:
        futures = []
        for iterable in iterables:
            futures += [ex.submit(fn, i) for i in iterable]
        for f in tqdm(as_completed(futures), total=len(futures), **kwargs):
            yield f.result()

### Discover regions where Azure AI is supported using the Models API

Not all Azure regions support Azure AI, let's filter out not supported regions

In [ ]:
def get_models_response(region):
    url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"
    return (region, requests.get(url, headers=headers))

def find_azureai_supported_regions():
    regions_successful = []
    regions_failed = []
    
    for (region, result) in parallel_map(get_models_response, regions_list):
        if result.status_code == 200:
            regions_successful.append(region)
        else:
            regions_failed.append(region)
    return (regions_successful, regions_failed)

regions_successful, regions_failed = find_azureai_supported_regions()
print("Potential Azure OpenAI regions based on control plane response:")
print(", ".join(regions_successful))

In [ ]:
print("Azure OpenAI Not Supported:")
print(", ".join(regions_failed))

## Models API: Discover models and SKUs

### List models

In [ ]:
def get_models(region):
    (_, response) = get_models_response(region)
    parsed = json.loads(response.text)
    return parsed['value']

def get_models_regions(regions):
    def get_models_region(region):
        return [{
            "model": model,
            "region": region
        } for model in get_models(region)]

    import itertools
    return itertools.chain.from_iterable(parallel_map(get_models_region, regions))

models_regions = list(get_models_regions(regions_successful))

In [ ]:
def format_model_region(model_region):
    model = model_region['model']
    return {
        'region': model_region['region'],
        'kind': model['kind'],
        'name': model['model']['name'],
        'version': model['model']['version'],
        'skus': ', '.join([sku['name'] for sku in model['model']['skus']]),
    }

## Convert the list to a DataFrame
df = pd.DataFrame(map(format_model_region, models_regions))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
df.head()

### List models by region

In [ ]:
region_model_data = {}  

excluded_models =  ['text-similarity-ada-001', 'text-babbage-001', 'text-curie-001', 'text-similarity-curie-001', 'text-davinci-002','text-davinci-003', 'text-davinci-fine-tune-002', 'code-davinci-002', 'code-davinci-fine-tune-002','text-ada-001', 'text-search-ada-doc-001', 'text-search-ada-query-001', 'code-search-ada-code-001','code-search-ada-text-001', 'text-similarity-babbage-001', 'text-search-babbage-doc-001','text-search-babbage-query-001', 'code-search-babbage-code-001', 'code-search-babbage-text-001', 'text-search-curie-doc-001', 'text-search-curie-query-001', 'text-davinci-001','text-similarity-davinci-001', 'text-search-davinci-doc-001', 'text-search-davinci-query-001','code-cushman-001']

for region in regions_successful:
    data_test = []

    for item in get_models(region):
        model_name = None
        version = None
        sku_name = None
        if item["model"]["capabilities"].get("scaleType") == "Manual": #skip legacy models
            continue
        model_name = item["model"]["name"]
        if model_name in excluded_models: # if in list skip
            continue
        version = item["model"]["version"]
        rdate = item["model"]["deprecation"]
        for sku in item["model"]["skus"]:
            sku_name = sku["name"]
        if sku_name == "Standard": # This example is only targeting Standard Model deployments SKUI
            data_test.append({"Model Name": model_name, "Version": version, "SKU Name": sku_name})
                #print(data_test)

    region_model_data[region] = data_test  # store the model data under corresponding region name

In [ ]:
rows = []
for region, models in region_model_data.items():
    for model in models:
        row = model.copy()  
        row['Region'] = region  
        rows.append(row)

df = pd.DataFrame(rows)
df = df[['Region', 'Model Name', 'Version', 'SKU Name']]
pd.set_option('display.max_rows', None)

df['Exist'] = True 
pivot_df = df.pivot_table(index='Region', columns=['Model Name', 'Version'], values='Exist', fill_value=False, aggfunc='any')
pivot_df.reset_index(inplace = True)

pivot_df

## Usages API: Discover quotas

In [ ]:
def get_usages(region):
    url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/usages?api-version=2023-05-01"
    response = requests.get(url, headers=headers)
    return json.loads(response.text)['value']

### Example for eastus2

In [ ]:
usages = get_usages('eastus2')

In [ ]:
def format_usage(usage):
    return {
        'name': usage['name']['value'],
        'localizedName': usage['name']['localizedValue'],
        'currentValue': usage['currentValue'],
        'limit': usage['limit'],
        'unit': usage['unit'],
    }

## Convert the list to a DataFrame
df = pd.DataFrame(map(format_usage, usages))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
df.head(10)

### Discover usages for all regions

In [ ]:
def get_regions_usages(regions):
    import itertools
    def get_region_usages(region):
        return (region, get_usages(region))
    return dict(parallel_map(get_region_usages, regions))

regional_usages = get_regions_usages(regions_successful)

### Index usages by name and region

We will use this index to join usages with the models on the **region** and **usage name**.

In [ ]:
def index_usages_by_name(usages):
    from functools import reduce
    def usage_name_reducer(j, u):
        j[u['name']['value']] = u
        return j
    return reduce(usage_name_reducer, usages, {})

regional_usages_by_name = dict([(region, index_usages_by_name(usages)) for (region, usages) in regional_usages.items()])

## Stitch APIs together

### Flatten models with SKUs

SKUs are listed as an array under each model. We want one SKU per line

In [ ]:
def flatten_models_sku(models_region):
    for model_join in models_region:
        model = model_join['model']
        skus = model['model']['skus']
        if skus:
            for sku in skus:
                yield model_join | {
                    "sku": sku
                }
model_region_skus = list(flatten_models_sku(models_regions))

In [ ]:
def format_model_region_sku(model_region_sku):
    sku = model_region_sku['sku']
    return format_model_region(model_region_sku) | {
        'skuName': sku['name'],
        'usageName': sku['usageName'],
    }

## Convert the list to a DataFrame
df = pd.DataFrame(map(format_model_region_sku, model_region_skus))
df.loc[:, df.columns != 'skus'].head()

> IMPORTANT: The `usageName` is the value we'll use to join models with usages

### Join models with usages

Use the **region** and the **SKU**'s `usageName` to join models with usages

In [ ]:
def join_models_usages(models_flattened, regional_usages_by_name):
    for model_join in models_flattened:
        model = model_join['model']
        sku = model_join['sku']
        region = model_join['region']
        usage = None
        if sku:
            usageName = sku['usageName']
            usages_by_name = regional_usages_by_name[region]
            usage = usages_by_name[usageName]
        yield model_join | {
            "usage": usage
        }

joined_model_sku_usages = list(join_models_usages(model_region_skus, regional_usages_by_name))

In [ ]:
def format_model_region_usage(model_region_usage):
    usage = model_region_usage['usage']
    return format_model_region_sku(model_region_usage) | format_usage(usage)

## Convert the list to a DataFrame
df = pd.DataFrame(map(format_model_region_usage, joined_model_sku_usages))
df.loc[:, df.columns != 'skus'].head()

## Find models matching requirements

### Find models matching name, TPM and SKU requirements

Let's define a helper function to help us filter models by name, remaining tpm and SKU:

In [ ]:
def model_matches(model_names, tpm, sku_names):
    def criteria(model, sku, usage, **kwargs):
        if not usage:
            return False
        limit = usage['limit']
        current = usage['currentValue']
        remaining = limit - current
        model_name = model['model']['name']
        sku_name = sku['name']
        matches = True
        if model_names:
            matches = matches and model_name in model_names
        if sku_names:
            matches = matches and sku_name in sku_names
        if tpm:
            matches = matches and remaining >= tpm
        return matches
    return criteria

def __or__(p1, p2):
    def pred(*args, **kwargs):
        return p1(*args, **kwargs) or p2(*args, **kwargs)
    return pred

def filter_models_sku_usages(criteria, models_sku_usages):
    return list(filter(lambda msu: criteria(**msu), models_sku_usages))

Let's take the example for the contoso-creative-writer sample Azure AI application:

| Usage	| Model	| Type	| TPM |
| --- | --- | --- | --- |
| editor	| gpt-35-turbo or gpt-35-turbo-16k	| Standard or Global Standard	| 10k |
| eval	| gpt-4 or gpt-4-32k	| Standard or Global Standard	| 20k |
| writer	| gpt-4o	| Standard or Global Standard	| 15k |
| embeddings	| text-embedding-3-small or text-embedding-ada-002	| Standard or Global Standard	| 30k |


Here's how to encode those requirements using the previous `model_matches` function:

In [ ]:
editor_req = model_matches(model_names = ['gpt-35-turbo', 'gpt-35-turbo-16k'], tpm = 10, sku_names = ['Standard', 'GlobalStandard'])
eval_req = model_matches(model_names = ['gpt-4', 'gpt-4-32k'], tpm = 5, sku_names = ['Standard', 'GlobalStandard'])
writer_req = model_matches(model_names = ['gpt-4o'], tpm = 15, sku_names = ['Standard', 'GlobalStandard'])
embedding_req = model_matches(model_names = ['text-embedding-3-small', 'text-embedding-ada-002'], tpm = 30, sku_names = ['Standard', 'GlobalStandard'])

requirements = [editor_req, eval_req, writer_req, embedding_req]

### Find all models matching requirements

Let's say you want to get an overview of all models matching any of the requirements

In [ ]:
req = __or__(editor_req, embedding_req)

models_sku_usages_filtered = filter_models_sku_usages(req, joined_model_sku_usages)

df = pd.DataFrame(map(format_model_region_usage, models_sku_usages_filtered))
df.loc[:, df.columns != 'skus'].head()

## Finding regions matching all model requirements

Sometimes, you're interested in deploying an AI application in a region where all requirements are met.

In [ ]:
def unique_regions(model_sku_usages):
    return set([model_sku_usage['region'] for model_sku_usage in model_sku_usages])

In [ ]:
def regions_matching_all(requirements, joined_model_sku_usages):
    filter_msu_sets = [filter_models_sku_usages(req, joined_model_sku_usages) for req in requirements]
    regions_sets = [unique_regions(model_sku_usages) for model_sku_usages in filter_msu_sets]
    return set.intersection(*regions_sets)

final_regions = regions_matching_all(requirements, joined_model_sku_usages)
print(", ".join(final_regions))

## Useful links

- [Azure REST API Browser](https://learn.microsoft.com/en-us/rest/api/)
- [Azure Locations API MS Learn](https://learn.microsoft.com/en-us/rest/api/resources/subscriptions/list-locations)
- [Azure AI Models API MS Learn](https://learn.microsoft.com/en-us/rest/api/aiservices/accountmanagement/models/list)
- [Azure AI Usages API MS Learn](https://learn.microsoft.com/en-us/rest/api/aiservices/accountmanagement/usages/list) (Quotas)